In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import QuickDrawDataset
from utils import AbsolutePenPositionTokenizer
from tqdm import tqdm

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)

Using device: cuda


In [11]:
labels = ["cat"]

training_data = QuickDrawDataset(
    labels=labels,
)

tokenizer = AbsolutePenPositionTokenizer(bins=64)

class SketchDataset(Dataset):
    def __init__(self, svg_list, tokenizer, max_len=200):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]

        for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
            tokens = tokenizer.encode(svg)
            # Truncate + Pad
            tokens = tokens[:max_len]
            tokens = tokens + [self.pad_id] * (max_len - len(tokens))
            self.data.append(tokens)

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids
    
    def __len__(self):
        return len(self.data)
    
dataset = SketchDataset(training_data, tokenizer, max_len=200)

Tokenizing SVGs:   6%|▌         | 6415/103031 [00:02<00:35, 2753.61it/s]


KeyboardInterrupt: 

In [9]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()

class SketchTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=6, max_len=200):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4*d_model
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch, seq_len) input tokens
        Returns: (batch, seq_len, vocab_size) logits
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        x = self.embed(x) + self.pos_embed(positions)  # (batch, seq_len, d_model)
        x = x.transpose(0, 1)  # -> (seq_len, batch, d_model)

        # causal mask (seq_len, seq_len)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)

        x = self.transformer(x, mask=mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)

        logits = self.fc_out(x)  # (batch, seq_len, vocab_size)
        return logits
    
def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignore pad token

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            # Transformer expects shape (seq_len, batch, d_model)
            logits = model(input_ids)  # (seq_len, batch, vocab_size)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
model = SketchTransformer(vocab_size=len(tokenizer.vocab), d_model=256, nhead=8, num_layers=6)
train_model(model, dataloader, vocab_size=len(tokenizer.vocab), epochs=40, lr=1e-4, device=device)

NameError: name 'dataset' is not defined

In [13]:
# save the model
torch.save(model, "sketch_transformer.pth")

In [ ]:
# model = torch.load("sketch_transformer.pth", map_location=device, weights_only=False)

def sample_sequence(model, start_token, max_len=200, temperature=1.0, greedy=False, eos_id=None, device="cuda"):
    model.eval()
    
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)  # (1, 1)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)  # (batch=1, seq_len, vocab_size)
            next_logits = logits[:, -1, :] / temperature  # take last step
            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        # stop if EOS reached
        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence(
    model, 
    start_token, 
    max_len=200, 
    temperature=0.5, 
    greedy=False, 
    eos_id=eos_token, 
    device=device
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(HTML(f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'))

Generated token sequence: [4098, 4096, 910, 843, 897, 960, 1088, 1152, 1153, 1354, 4096, 1418, 1610, 2056, 2440, 2569, 4096, 2894, 3209, 3461, 3846, 3717, 3786, 3856, 4096, 845, 657, 790, 796, 868, 1063, 1324, 1647, 2225, 2670, 3050, 3236, 3359, 3414, 4096, 1489, 1364, 1497, 4096, 2450, 2456, 4096, 2012, 1950, 1824, 1824, 2016, 2080, 2079, 4096, 2014, 1825, 1635, 1636, 4096, 2015, 2019, 2148, 2469, 4096, 2080, 2082, 2149, 2277, 4096, 2783, 4061, 4096, 2784, 3747, 4096, 2788, 2987, 4096, 992, 477, 26, 4096, 1188, 677, 4096, 1190, 811, 361, 4096, 1257, 942, 4099]
Decoded sketch: <svg viewBox="0 0 64 64"><g stroke-width="0.8">
<path d="M 14 14 L 13 11 L 14 1 L 15 0 L 17 0 L 18 0 L 18 1 L 21 10" stroke="black" fill="none"/>
<path d="M 22 10 L 25 10 L 32 8 L 38 8 L 40 9" stroke="black" fill="none"/>
<path d="M 45 14 L 50 9 L 54 5 L 60 6 L 58 5 L 59 10 L 60 16" stroke="black" fill="none"/>
<path d="M 13 13 L 10 17 L 12 22 L 12 28 L 13 36 L 16 39 L 20 44 L 25 47 L 34 49 L 41 46 L 47 42 L 50 3